### Psycopg2

1. Wczytaj dane z pliku `cars.csv` i wyodrębnij z niego kolumny:
   - `price`
   - `brand`
   - `fuel`
   - `power`
   - `prod_year`


Następnie napisz zapytanie `CREATE`, które utworzy tabelę o takim samym schemacie + dodatkowa kolumna `id` typu `serial`. Wykonaj zapytanie za pomocą `psycopg2`

In [12]:
import pandas as pd
import psycopg2

df = pd.read_csv("cars.csv", usecols=["price", "brand", "fuel", "power", "prod_year"]).dropna()
df

,price,brand,fuel,power,prod_year
0,32900,Škoda,Diesel,115.0,2017
1,32500,BMW,Diesel,116.0,2012
2,7900,Fiat,Benzyna,69.0,2012
3,39990,Kia,Diesel,115.0,2012
4,42900,Kia,Diesel,115.0,2012
...,...,...,...,...,...
99995,83990,Ford,Diesel,120.0,2020
99996,7400,Audi,Benzyna,125.0,1999
99997,30000,BMW,Diesel,245.0,1999
99998,69000,Peugeot,Benzyna,101.0,2020


In [11]:
create_query = """CREATE TABLE cars(
    id SERIAL,
    price INTEGER,
    brand TEXT,
    fuel TEXT,
    power SMALLINT,
    prod_year SMALLINT
)
"""

In [13]:
conn = psycopg2.connect(dbname="postgres", user="postgres", password="postgres", host="localhost")
cursor = conn.cursor()

In [14]:
cursor.execute(create_query)
conn.commit()

2. Napisz kod, który na podstawie numeru indeksu wiersza dataframe'a, wyciągnie ten wiersz z tabeli, utworzy zapytanie `INSERT` a następnie wykona je na bazie.

In [6]:
import numpy as np

In [7]:
def get_row_from_df(df, idx):
    return df.loc[idx]

In [8]:
def get_values_from_record(record):
    return [x.item() if isinstance(x, np.generic) else x for x in record.values]

In [9]:
def generate_insert_query_template(record):
    return f"INSERT INTO cars(price, brand, fuel, power, prod_year) VALUES ({','.join(['%s']*len(record))})"

In [10]:
def run_query(query_template, values):
    cursor.execute(query_template, values)
    conn.commit()

In [13]:
record = get_row_from_df(df, 0)
values = get_values_from_record(record)

In [14]:
query = generate_insert_query_template(record)
query

'INSERT INTO cars(price, brand, fuel, power, prod_year) VALUES (%s,%s,%s,%s,%s)'

In [16]:
run_query(query, values)

NameError: name 'cursor' is not defined

3. Napisz funkcję, która przyjmie zapytanie typu `SELECT` i wykona je na bazie.

In [7]:
# ...

4. Napisz funkcję, która przyjmie `id` wiersza w tabeli, nazwę kolumny oraz nową wartość a następnie zupdatuje tabelę.

In [5]:
# ...

5. Napisz funkcję, która przyjmie `id` wiersza w tabeli a następnie go usunie.

In [6]:
# ...

### SQLAlchemy

1. Utwórz klasę odpowiadającą tabeli `cars` a następnie samą tabelę za pomocą `SQLAlchemy`.

In [1]:
from sqlalchemy import Column, Integer, Boolean, VARCHAR, String
from sqlalchemy.sql.expression import false
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, declarative_base

In [2]:
user = "postgres"
password = "postgres"
host = "localhost"
port = "5432"
database = "postgres"

Base = declarative_base()
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)
Session = sessionmaker(bind=engine)
session = Session()

In [3]:
class CarsTable(Base):
    __tablename__ = "cars"

    id = Column("id", Integer, primary_key=True)
    price = Column("price", Integer)
    brand = Column("brand", String)
    fuel = Column("fuel", String)
    power = Column("power", Integer)
    prod_year = Column("prod_year", Integer)

    def __repr__(self):
        return f"Cars(id={self.id})"

In [4]:
Base.metadata.create_all(bind=engine)

2. Napisz kod, który doda do tabeli wiersz z dataframe'a wykorzystując `SQLAlchemy`.

In [46]:
record = get_row_from_df(df, 5)
record.values[:] = get_values_from_record(record)

In [47]:
car = CarsTable(**record)

In [48]:
session.add(car)
session.commit()

In [35]:
# car.id

3. Napisz kod, który wyciągnie dane z dataframe'a na różne sposoby

In [50]:
session.query(CarsTable).all()

[Cars(id=1), Cars(id=2), Cars(id=3), Cars(id=4), Cars(id=5), Cars(id=6)]

In [54]:
session.query(CarsTable.price, CarsTable.brand).all()

[(32900, 'Škoda'),
 (32500, 'BMW'),
 (7900, 'Fiat'),
 (39990, 'Kia'),
 (42900, 'Kia'),
 (19400, 'BMW')]

In [61]:
from sqlalchemy import desc, gre

In [58]:
session.query(CarsTable).order_by(desc(CarsTable.power)).all()

[Cars(id=6), Cars(id=2), Cars(id=1), Cars(id=4), Cars(id=5), Cars(id=3)]

In [62]:
session.query(CarsTable).filter(CarsTable.prod_year > 2010).all()

[Cars(id=1), Cars(id=2), Cars(id=3), Cars(id=4), Cars(id=5)]